# Working with Complex Data Types in SQL - Interactive Demo

Welcome! This demo will teach you how to work with complex data types using Databricks SQL.

---

## 📊 What are Complex Data Types?

**Complex data types** allow you to store nested and hierarchical data structures:

* **STRUCT** - Nested objects with named fields (like JSON objects)
* **ARRAY** - Ordered collections of elements (like JSON arrays)
* **MAP** - Key-value pairs
* **JSON** - Semi-structured text data

**Why use SQL for complex types?**
* Familiar SQL syntax for nested data
* No need to switch to Python/Scala
* Works in SQL warehouses and notebooks
* Perfect for BI tools and dashboards

---

## 🎯 What You'll Learn

1. **STRUCT in SQL** - Access nested fields with dot notation
2. **ARRAY in SQL** - Work with collections using SQL functions
3. **EXPLODE** - Flatten arrays into rows
4. **Array Functions** - Filter, transform, and aggregate arrays
5. **JSON Operations** - Parse and generate JSON in SQL
6. **Real-World Patterns** - Common use cases and best practices

---

## 📚 Demo Dataset

We'll use the **Wanderbricks** sample data:
* **clickstream** - User events with struct metadata
* **customer_support_logs** - Support tickets with array of messages

**Let's get started!** 🚀

## 1. Working with STRUCT in SQL 📊

**What is a STRUCT?**

A STRUCT is a nested data type that contains multiple named fields, like a JSON object.

**Example:**
```sql
metadata: STRUCT<device: STRING, referrer: STRING>
```

**Common use cases:**
* API responses with nested objects
* Event metadata
* Address information (street, city, state, zip)
* User profiles with multiple attributes

Let's explore the **clickstream** table which has a struct column!

In [0]:
%sql
-- View the schema of the clickstream table
DESCRIBE samples.wanderbricks.clickstream

In [0]:
%sql
-- Let's see what the struct data looks like
SELECT *
FROM samples.wanderbricks.clickstream
LIMIT 10

In [0]:
%sql
-- Access nested struct fields using dot notation
-- This is the most common approach in SQL

SELECT 
  event,
  user_id,
  property_id,
  metadata.device AS device,        -- Access nested field
  metadata.referrer AS referrer,    -- Access nested field
  timestamp
FROM samples.wanderbricks.clickstream
LIMIT 10

In [0]:
%sql
-- Filter based on nested struct fields
-- Find all mobile events from Google

SELECT 
  event,
  user_id,
  metadata.device AS device,
  metadata.referrer AS referrer,
  timestamp
FROM samples.wanderbricks.clickstream
WHERE 
  metadata.device = 'mobile'
  AND metadata.referrer = 'google'
LIMIT 20

In [0]:
%sql
-- Group by nested struct fields
-- Analyze events by device and referrer

SELECT 
  metadata.device AS device,
  metadata.referrer AS referrer,
  COUNT(*) AS event_count
FROM samples.wanderbricks.clickstream
GROUP BY metadata.device, metadata.referrer
ORDER BY event_count DESC

In [0]:
%sql
-- Use .* to expand all fields from a struct
-- This flattens the struct into separate columns

SELECT 
  event,
  user_id,
  metadata.*,  -- Expands to: device, referrer
  timestamp
FROM samples.wanderbricks.clickstream
LIMIT 10

In [0]:
%sql
-- Create a new struct using STRUCT() or named_struct()

SELECT 
  event,
  user_id,
  -- Create a new struct with selected fields
  STRUCT(
    metadata.device AS device,
    metadata.referrer AS referrer,
    'clickstream' AS source
  ) AS event_summary
FROM samples.wanderbricks.clickstream
LIMIT 10

### ✅ STRUCT Best Practices in SQL

**Accessing Fields:**
```sql
-- Dot notation (recommended)
SELECT struct_column.field_name FROM table

-- Expand all fields
SELECT struct_column.* FROM table
```

**Filtering:**
```sql
-- Filter on nested fields
WHERE metadata.device = 'mobile'
```

**Grouping:**
```sql
-- Group by nested fields
GROUP BY metadata.device, metadata.referrer
```

**Creating Structs:**
```sql
-- Using STRUCT()
SELECT STRUCT(field1, field2 AS renamed) AS new_struct

-- Using named_struct()
SELECT named_struct('key1', value1, 'key2', value2) AS new_struct
```

**💡 Pro Tip:** Structs keep related data together without creating separate columns!

## 2. Working with ARRAY in SQL 📊

**What is an ARRAY?**

An ARRAY is an ordered collection of elements of the same type.

**Example:**
```sql
messages: ARRAY<STRUCT<message: STRING, sender: STRING, sentiment: STRING, timestamp: STRING>>
```

**Common use cases:**
* Lists of items (tags, categories, products)
* Time-series events
* Message threads
* Transaction histories

Let's explore the **customer_support_logs** table which has an array of messages!

In [0]:
%sql
-- View the schema of the customer support logs table
DESCRIBE samples.wanderbricks.customer_support_logs

In [0]:
%sql
-- Let's see what the array data looks like
-- Each ticket has an array of messages

SELECT 
  ticket_id,
  user_id,
  support_agent_id,
  messages
FROM samples.wanderbricks.customer_support_logs
LIMIT 3

In [0]:
%sql
-- Get the number of elements in an array using SIZE() or ARRAY_SIZE()

SELECT 
  ticket_id,
  user_id,
  SIZE(messages) AS message_count,
  messages
FROM samples.wanderbricks.customer_support_logs
LIMIT 10

In [0]:
%sql
-- Access specific array elements using bracket notation [index]
-- Arrays are 0-indexed (first element is [0])

SELECT 
  ticket_id,
  messages[0] AS first_message,     -- First message (entire struct)
  messages[1] AS second_message     -- Second message
FROM samples.wanderbricks.customer_support_logs
LIMIT 5

In [0]:
%sql
-- Access fields within array elements
-- Combine array indexing with struct field access

SELECT 
  ticket_id,
  messages[0].message AS first_message_text,
  messages[0].sender AS first_sender,
  messages[0].sentiment AS first_sentiment,
  messages[0].timestamp AS first_timestamp
FROM samples.wanderbricks.customer_support_logs
LIMIT 10

In [0]:
%sql
-- Find tickets with many messages (long conversations)

SELECT 
  ticket_id,
  user_id,
  support_agent_id,
  SIZE(messages) AS message_count
FROM samples.wanderbricks.customer_support_logs
WHERE SIZE(messages) >= 4
ORDER BY message_count DESC

In [0]:
%sql
-- Compare the first and last message in each conversation
-- This shows how sentiment changes over the conversation

SELECT 
  ticket_id,
  SIZE(messages) AS total_messages,
  messages[0].sender AS first_sender,
  messages[0].sentiment AS first_sentiment,
  messages[SIZE(messages) - 1].sender AS last_sender,
  messages[SIZE(messages) - 1].sentiment AS last_sentiment
FROM samples.wanderbricks.customer_support_logs
WHERE SIZE(messages) >= 2
LIMIT 10

## 3. EXPLODE and Array Functions 💥

**What is EXPLODE?**

EXPLODE transforms an array into multiple rows - one row per array element.

**Before EXPLODE:**
```
ticket_id | messages (array with 3 elements)
```

**After EXPLODE:**
```
ticket_id | message (row 1)
ticket_id | message (row 2)
ticket_id | message (row 3)
```

**Why use EXPLODE?**
* Flatten nested data for analysis
* Join array elements with other tables
* Aggregate across all array elements
* Make nested data queryable

Let's see it in action!

In [0]:
%sql
-- EXPLODE creates one row per array element
-- This flattens the messages array

SELECT 
  ticket_id,
  user_id,
  EXPLODE(messages) AS message
FROM samples.wanderbricks.customer_support_logs
LIMIT 20

In [0]:
%sql
-- After EXPLODE, you can access struct fields from the exploded elements
-- This gives you one row per message with all message details

SELECT 
  ticket_id,
  user_id,
  exploded_message.message AS message_text,
  exploded_message.sender AS sender,
  exploded_message.sentiment AS sentiment,
  exploded_message.timestamp AS message_timestamp
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW EXPLODE(messages) AS exploded_message
LIMIT 20

In [0]:
%sql
-- Alternative: Use inline EXPLODE (cleaner syntax)
-- This is the modern Databricks SQL approach

SELECT 
  ticket_id,
  user_id,
  message.message AS message_text,
  message.sender AS sender,
  message.sentiment AS sentiment,
  message.timestamp AS message_timestamp
FROM samples.wanderbricks.customer_support_logs,
  LATERAL VIEW EXPLODE(messages) AS message
LIMIT 20

In [0]:
%sql
-- Use EXPLODE to analyze all messages across all tickets
-- Count messages by sender and sentiment

SELECT 
  message.sender AS sender,
  message.sentiment AS sentiment,
  COUNT(*) AS message_count
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW EXPLODE(messages) AS message
GROUP BY message.sender, message.sentiment
ORDER BY message_count DESC

In [0]:
%sql
-- EXPLODE_OUTER keeps rows even if the array is empty or NULL
-- Regular EXPLODE drops rows with empty/null arrays

SELECT 
  ticket_id,
  user_id,
  SIZE(messages) AS message_count,
  message.sender AS sender
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW EXPLODE_OUTER(messages) AS message
LIMIT 20

### 🛠️ Common Array Functions in SQL

**Size and Existence:**
* `SIZE(array)` or `ARRAY_SIZE(array)` - Number of elements
* `ARRAY_CONTAINS(array, value)` - Check if value exists
* `ARRAY_POSITION(array, value)` - Find index of value

**Transformation:**
* `EXPLODE(array)` - One row per element
* `EXPLODE_OUTER(array)` - Keeps rows with empty arrays
* `POSEXPLODE(array)` - Explode with position index

**Manipulation:**
* `ARRAY_DISTINCT(array)` - Remove duplicates
* `ARRAY_SORT(array)` - Sort elements
* `ARRAY_UNION(array1, array2)` - Combine arrays
* `ARRAY_INTERSECT(array1, array2)` - Common elements
* `ARRAY_EXCEPT(array1, array2)` - Elements in array1 not in array2

**Filtering:**
* `FILTER(array, x -> condition)` - Filter array elements
* `ARRAY_REMOVE(array, value)` - Remove specific value

**Aggregation:**
* `ARRAY_MIN(array)` - Minimum value
* `ARRAY_MAX(array)` - Maximum value
* `ARRAY_JOIN(array, delimiter)` - Join to string

In [0]:
%sql
-- Find tickets where any message has 'angry' sentiment
-- Using ARRAY_CONTAINS to search within arrays

SELECT 
  ticket_id,
  user_id,
  SIZE(messages) AS message_count
FROM samples.wanderbricks.customer_support_logs
WHERE EXISTS (
  SELECT 1 
  FROM LATERAL VIEW EXPLODE(messages) AS msg
  WHERE msg.sentiment = 'angry'
)
LIMIT 10

In [0]:
%sql
-- POSEXPLODE gives you both the position (index) and the element
-- Useful for tracking message order in conversations

SELECT 
  ticket_id,
  pos AS message_number,
  message.sender AS sender,
  message.sentiment AS sentiment,
  message.message AS message_text
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW POSEXPLODE(messages) AS pos, message
WHERE ticket_id = 'TICKET-000001'
ORDER BY pos

In [0]:
%sql
-- FILTER function to filter array elements inline
-- Get only messages from users (not agents)

SELECT 
  ticket_id,
  messages AS all_messages,
  FILTER(messages, msg -> msg.sender = 'user') AS user_messages_only,
  SIZE(FILTER(messages, msg -> msg.sender = 'user')) AS user_message_count
FROM samples.wanderbricks.customer_support_logs
LIMIT 5

## 4. JSON Operations in SQL 📝

**Working with JSON in Databricks SQL:**

* **TO_JSON()** - Convert struct/array to JSON string
* **FROM_JSON()** - Parse JSON string to struct/array
* **GET_JSON_OBJECT()** - Extract value from JSON string
* **JSON_TUPLE()** - Extract multiple values from JSON

**Common scenarios:**
* API responses stored as JSON strings
* Converting complex types for export
* Parsing semi-structured data
* Interoperability with external systems

In [0]:
%sql
-- Convert struct to JSON string using TO_JSON()
-- Useful for exporting or storing as text

SELECT 
  event,
  user_id,
  metadata,                          -- Original struct
  TO_JSON(metadata) AS metadata_json -- Converted to JSON string
FROM samples.wanderbricks.clickstream
LIMIT 10

In [0]:
%sql
-- Convert array to JSON string
-- This serializes the entire messages array

SELECT 
  ticket_id,
  user_id,
  SIZE(messages) AS message_count,
  TO_JSON(messages) AS messages_json
FROM samples.wanderbricks.customer_support_logs
LIMIT 3

In [0]:
%sql
-- FROM_JSON parses a JSON string into a struct
-- You must specify the schema

SELECT 
  event,
  user_id,
  TO_JSON(metadata) AS json_string,
  FROM_JSON(
    TO_JSON(metadata),
    'STRUCT<device: STRING, referrer: STRING>'
  ) AS parsed_struct
FROM samples.wanderbricks.clickstream
LIMIT 5

In [0]:
%sql
-- GET_JSON_OBJECT extracts a value from a JSON string
-- Useful when you have JSON stored as STRING

SELECT 
  event,
  user_id,
  TO_JSON(metadata) AS json_string,
  GET_JSON_OBJECT(TO_JSON(metadata), '$.device') AS device,
  GET_JSON_OBJECT(TO_JSON(metadata), '$.referrer') AS referrer
FROM samples.wanderbricks.clickstream
LIMIT 10

### 📍 JSON Path Expressions

**GET_JSON_OBJECT uses JSON path syntax:**

```sql
-- Root level
GET_JSON_OBJECT(json_col, '$.field_name')

-- Nested objects
GET_JSON_OBJECT(json_col, '$.parent.child')

-- Array elements
GET_JSON_OBJECT(json_col, '$.array[0]')

-- Nested array elements
GET_JSON_OBJECT(json_col, '$.array[0].field')
```

**Examples:**
```sql
-- Extract device from metadata JSON
GET_JSON_OBJECT(metadata_json, '$.device')

-- Extract first message text
GET_JSON_OBJECT(messages_json, '$[0].message')

-- Extract nested field
GET_JSON_OBJECT(data_json, '$.user.profile.name')
```

In [0]:
%sql
-- Practical example: Parse JSON string and extract multiple fields
-- Simulating data that comes from an API as JSON

WITH json_data AS (
  SELECT 
    ticket_id,
    TO_JSON(messages[0]) AS first_message_json
  FROM samples.wanderbricks.customer_support_logs
)
SELECT 
  ticket_id,
  first_message_json,
  GET_JSON_OBJECT(first_message_json, '$.message') AS message_text,
  GET_JSON_OBJECT(first_message_json, '$.sender') AS sender,
  GET_JSON_OBJECT(first_message_json, '$.sentiment') AS sentiment
FROM json_data
LIMIT 10

## 5. Real-World Patterns 🎯

Let's combine what we've learned to solve real business problems!

In [0]:
%sql
-- Real-world pattern: Analyze customer support conversations
-- Combine EXPLODE with aggregations to understand sentiment trends

SELECT 
  message.sender AS sender,
  message.sentiment AS sentiment,
  COUNT(*) AS message_count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY message.sender), 2) AS percentage
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW EXPLODE(messages) AS message
GROUP BY message.sender, message.sentiment
ORDER BY sender, message_count DESC

In [0]:
%sql
-- Real-world pattern: Track how sentiment changes from start to end
-- Compare first user message with last agent message

SELECT 
  ticket_id,
  user_id,
  SIZE(messages) AS total_messages,
  messages[0].sentiment AS initial_sentiment,
  messages[SIZE(messages) - 1].sentiment AS final_sentiment,
  CASE 
    WHEN messages[0].sentiment = 'angry' AND messages[SIZE(messages) - 1].sentiment = 'caring' 
      THEN 'Resolved Positively'
    WHEN messages[0].sentiment = messages[SIZE(messages) - 1].sentiment 
      THEN 'No Change'
    ELSE 'Changed'
  END AS sentiment_change
FROM samples.wanderbricks.customer_support_logs
WHERE SIZE(messages) >= 2
LIMIT 20

In [0]:
%sql
-- Real-world pattern: Find tickets with angry customer messages
-- Use FILTER to get only user messages with angry sentiment

SELECT 
  ticket_id,
  user_id,
  support_agent_id,
  SIZE(messages) AS total_messages,
  SIZE(FILTER(messages, msg -> msg.sender = 'user' AND msg.sentiment = 'angry')) AS angry_user_messages,
  FILTER(messages, msg -> msg.sender = 'user' AND msg.sentiment = 'angry') AS angry_messages
FROM samples.wanderbricks.customer_support_logs
WHERE SIZE(FILTER(messages, msg -> msg.sender = 'user' AND msg.sentiment = 'angry')) > 0
LIMIT 10

In [0]:
%sql
-- Real-world pattern: Multi-dimensional analysis with structs
-- Analyze conversion funnel by device and referrer

SELECT 
  metadata.device AS device,
  metadata.referrer AS referrer,
  COUNT(DISTINCT user_id) AS unique_users,
  COUNT(*) AS total_events,
  SUM(CASE WHEN event = 'view' THEN 1 ELSE 0 END) AS views,
  SUM(CASE WHEN event = 'click' THEN 1 ELSE 0 END) AS clicks,
  ROUND(SUM(CASE WHEN event = 'click' THEN 1 ELSE 0 END) * 100.0 / 
        NULLIF(SUM(CASE WHEN event = 'view' THEN 1 ELSE 0 END), 0), 2) AS click_through_rate
FROM samples.wanderbricks.clickstream
GROUP BY metadata.device, metadata.referrer
ORDER BY total_events DESC

In [0]:
%sql
-- Real-world pattern: Flatten array data and join with other tables
-- Get user details for each message in support tickets

SELECT 
  t.ticket_id,
  u.name AS user_name,
  u.email AS user_email,
  msg.sender AS message_sender,
  msg.sentiment AS sentiment,
  msg.message AS message_text
FROM samples.wanderbricks.customer_support_logs t
LATERAL VIEW EXPLODE(t.messages) AS msg
JOIN samples.wanderbricks.users u ON t.user_id = u.user_id
WHERE msg.sender = 'user'
LIMIT 20

In [0]:
%sql
-- Real-world pattern: Create a flattened summary table
-- This is useful for BI tools that don't handle complex types well

CREATE OR REPLACE TABLE main.default.support_messages_flat AS
SELECT 
  ticket_id,
  user_id,
  support_agent_id,
  created_at AS ticket_created_at,
  msg_pos AS message_position,
  msg.message AS message_text,
  msg.sender AS sender,
  msg.sentiment AS sentiment,
  msg.timestamp AS message_timestamp
FROM samples.wanderbricks.customer_support_logs
LATERAL VIEW POSEXPLODE(messages) AS msg_pos, msg

In [0]:
%sql
-- Verify the flattened table was created
SELECT *
FROM main.default.support_messages_flat
ORDER BY ticket_id, message_position
LIMIT 20

## 📚 Quick Reference Guide

### **STRUCT Operations**
```sql
-- Access fields
SELECT struct_col.field_name FROM table

-- Expand all fields
SELECT struct_col.* FROM table

-- Filter on nested field
WHERE struct_col.field_name = 'value'

-- Create struct
SELECT STRUCT(col1, col2 AS renamed) AS new_struct
```

### **ARRAY Operations**
```sql
-- Array size
SELECT SIZE(array_col) FROM table

-- Access by index (0-based)
SELECT array_col[0] FROM table

-- Access nested field in array
SELECT array_col[0].field_name FROM table

-- Last element
SELECT array_col[SIZE(array_col) - 1] FROM table
```

### **EXPLODE Operations**
```sql
-- Basic explode
SELECT id, EXPLODE(array_col) AS element FROM table

-- Explode with LATERAL VIEW
SELECT id, elem
FROM table
LATERAL VIEW EXPLODE(array_col) AS elem

-- Explode with position
SELECT id, pos, elem
FROM table
LATERAL VIEW POSEXPLODE(array_col) AS pos, elem

-- Explode keeping nulls
SELECT id, elem
FROM table
LATERAL VIEW EXPLODE_OUTER(array_col) AS elem
```

### **Array Functions**
```sql
SIZE(array)                          -- Number of elements
ARRAY_CONTAINS(array, value)         -- Check existence
FILTER(array, x -> condition)        -- Filter elements
ARRAY_DISTINCT(array)                -- Remove duplicates
ARRAY_SORT(array)                    -- Sort elements
ARRAY_JOIN(array, delimiter)         -- Join to string
```

### **JSON Operations**
```sql
TO_JSON(struct_or_array)             -- Convert to JSON string
FROM_JSON(json_string, schema)       -- Parse JSON to struct
GET_JSON_OBJECT(json_str, '$.path')  -- Extract value
```

## 📊 Common Patterns Summary

### **Pattern 1: Flatten for Analysis**
```sql
-- Explode arrays to analyze individual elements
SELECT id, element
FROM table
LATERAL VIEW EXPLODE(array_col) AS element
```

### **Pattern 2: Access Nested Data**
```sql
-- Use dot notation for structs, brackets for arrays
SELECT 
  struct_col.field,
  array_col[0].nested_field
FROM table
```

### **Pattern 3: Filter Array Contents**
```sql
-- Use FILTER to get subset of array
SELECT 
  id,
  FILTER(array_col, x -> x.status = 'active') AS active_items
FROM table
```

### **Pattern 4: Aggregate Across Arrays**
```sql
-- Explode then aggregate
SELECT 
  category,
  COUNT(*) AS total_items
FROM table
LATERAL VIEW EXPLODE(items) AS item
GROUP BY item.category
```

### **Pattern 5: Convert for Export**
```sql
-- Convert complex types to JSON for external systems
SELECT 
  id,
  TO_JSON(complex_col) AS json_data
FROM table
```

## ✅ Best Practices for Complex Data Types

### **1. Performance Considerations**

✅ **Use EXPLODE carefully** - Creates many rows, can be expensive  
✅ **Filter before EXPLODE** - Reduce data volume first  
✅ **Limit EXPLODE results** - Use LIMIT during development  
✅ **Index access is fast** - `array[0]` is efficient  
❌ **Avoid nested EXPLODE** - Can create cartesian products  

### **2. When to Use Complex Types**

✅ **Use STRUCT when:**
* Related fields belong together
* Data comes from APIs/JSON
* Want to reduce column count

✅ **Use ARRAY when:**
* Variable number of items per row
* Order matters
* Items are homogeneous

❌ **Don't use complex types when:**
* Data is naturally flat
* Need frequent filtering on nested fields
* BI tools don't support them

### **3. Flattening Strategy**

**For BI/Reporting:**
* Flatten complex types into separate tables
* Use CTAS to create materialized views
* Easier for analysts and BI tools

**For Processing:**
* Keep complex types for efficiency
* Use dot notation and array functions
* Flatten only when necessary

### **4. Schema Evolution**

✅ Add new struct fields without breaking queries  
✅ Use `.*` to automatically include new fields  
✅ Document complex type schemas  
❌ Be careful with array element schemas changing  

## 🎉 Congratulations!

You've completed the Complex Data Types in SQL demo!

### **What You Learned:**

✅ **STRUCT** - Access nested fields with dot notation  
✅ **ARRAY** - Work with collections using bracket notation  
✅ **EXPLODE** - Flatten arrays into rows for analysis  
✅ **Array Functions** - SIZE, FILTER, ARRAY_CONTAINS, and more  
✅ **JSON Operations** - TO_JSON, FROM_JSON, GET_JSON_OBJECT  
✅ **Real-World Patterns** - Practical use cases and combinations  

---

### **Key Takeaways:**

1. **Complex types are powerful** - Store hierarchical data efficiently
2. **SQL handles them well** - Dot notation and array functions make it easy
3. **EXPLODE is your friend** - Flatten when you need row-level analysis
4. **Performance matters** - Filter before exploding, use indexes
5. **Choose wisely** - Flatten for BI, keep nested for processing

---

### **Next Steps:**

* Practice with your own nested data
* Explore TRANSFORM and higher-order functions
* Learn about MAP data type
* Study semi-structured data patterns
* Build ETL pipelines with complex types

---

### **Resources:**

* [Databricks SQL Functions](https://docs.databricks.com/sql/language-manual/sql-ref-functions.html)
* [Complex Data Types Guide](https://docs.databricks.com/sql/language-manual/sql-ref-datatypes.html)
* [Array Functions Reference](https://docs.databricks.com/sql/language-manual/sql-ref-functions-builtin.html#array-functions)
* [JSON Functions Reference](https://docs.databricks.com/sql/language-manual/sql-ref-functions-builtin.html#json-functions)

---

**Pro Tip:** Master complex data types to work efficiently with modern data formats like JSON, Parquet, and Delta Lake! 🚀

*Happy querying!*